## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 15
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_15


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 24.8 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,14,2022,29.6,80.0,8.1,309.5,104.5,12.1,23.0,...,10.4,105.3,21.571429,11.714286,12.857143,0.072464,2.500000,0.571429,3.75,3.181818
1,Buffalo Bills,14,2022,27.5,50.0,7.5,265.7,96.4,11.8,17.9,...,10.5,106.1,18.857143,12.142857,13.071429,0.095745,2.142857,0.714286,1.70,2.727273


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[14]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_15,0.370846,0.333554,0.221764,0.338684,0.409601,0.334890,0.404757
1,Buffalo Bills,2022,week_15,0.420316,0.219178,0.130390,0.241718,0.364870,0.275295,0.371801
2,Philadelphia Eagles,2022,week_15,0.404552,0.265670,0.151194,0.264036,0.355796,0.288250,0.361890


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_15,0.370846,0.333554,0.221764,0.338684,0.409601,0.334890,0.404757
1,Buffalo Bills,2022,week_15,0.420316,0.219178,0.130390,0.241718,0.364870,0.275295,0.371801
2,Philadelphia Eagles,2022,week_15,0.404552,0.265670,0.151194,0.264036,0.355796,0.288250,0.361890
6,San Francisco 49ers,2022,week_15,0.318422,0.245168,0.130409,0.280411,0.362723,0.267427,0.357186
4,Miami Dolphins,2022,week_15,0.143905,0.270114,0.143712,0.286408,0.364745,0.241777,0.337140
8,Dallas Cowboys,2022,week_15,0.129700,0.085872,0.069395,0.194954,0.334508,0.162886,0.308907
9,Cincinnati Bengals,2022,week_15,0.041745,0.095105,0.058629,0.186855,0.345224,0.145512,0.307289
13,Seattle Seahawks,2022,week_15,0.007095,0.109064,0.057905,0.210169,0.350033,0.146853,0.307166
5,Jacksonville Jaguars,2022,week_15,0.004445,0.037162,0.044087,0.182621,0.337111,0.121085,0.295527
16,New Orleans Saints,2022,week_15,0.015580,0.085660,0.069737,0.209962,0.335287,0.143245,0.295323
